In [1]:
import torch
import torch.nn as nn
import numpy as np

# from utils.adv_trainer import *

import os
import sys
root = os.path.abspath('/home/mbeliaev/home/code/robust-l0/')
sys.path.append(root)

from utils.models import *
from utils.helpers import *
from utils.attack import *

from tqdm import trange
import pickle
import json

import torch

In [13]:
path_clip = '/home/mbeliaev/home/code/robust-l0/new_trained/beta_1/CIFAR/VGG16/clip'

subdirs_clip = [os.path.join(root, name)
             for root, dirs, files in os.walk(path_clip)
             for name in files
             if name.endswith((".json"))]

print(subdirs_clip)

path_simple = '/home/mbeliaev/home/code/robust-l0/new_trained/beta_1/CIFAR/VGG16/simple'

subdirs_simple = [os.path.join(root, name)
             for root, dirs, files in os.walk(path_simple)
             for name in files
             if name.endswith((".json"))]

print(subdirs_simple)



['/home/mbeliaev/home/code/robust-l0/new_trained/beta_1/CIFAR/VGG16/clip/k_12/adv_12/5/setup.json', '/home/mbeliaev/home/code/robust-l0/new_trained/beta_1/CIFAR/VGG16/clip/k_12/adv_12/3/setup.json', '/home/mbeliaev/home/code/robust-l0/new_trained/beta_1/CIFAR/VGG16/clip/k_12/adv_12/4/setup.json', '/home/mbeliaev/home/code/robust-l0/new_trained/beta_1/CIFAR/VGG16/clip/k_12/adv_12/2/setup.json', '/home/mbeliaev/home/code/robust-l0/new_trained/beta_1/CIFAR/VGG16/clip/k_12/adv_12/1/setup.json']
['/home/mbeliaev/home/code/robust-l0/new_trained/beta_1/CIFAR/VGG16/simple/k_12/adv_12/5/setup.json', '/home/mbeliaev/home/code/robust-l0/new_trained/beta_1/CIFAR/VGG16/simple/k_12/adv_12/3/setup.json', '/home/mbeliaev/home/code/robust-l0/new_trained/beta_1/CIFAR/VGG16/simple/k_12/adv_12/4/setup.json', '/home/mbeliaev/home/code/robust-l0/new_trained/beta_1/CIFAR/VGG16/simple/k_12/adv_12/2/setup.json', '/home/mbeliaev/home/code/robust-l0/new_trained/beta_1/CIFAR/VGG16/simple/k_12/adv_12/1/setup.json'

In [2]:
import pickle

result = pickle.load(open('/home/mbeliaev/home/code/robust-l0/new_trained/final_no/result_CIFAR_0_100.p','rb'))

In [53]:
print('Evaluating Clip')
acc_data = []
r_acc_data = []
for path in subdirs_clip:
    results = pickle.load(open(path[0:-len('setup.json')]+'results.p','rb'))
    f_results = pickle.load(open(path[0:-len('setup.json')]+'f_results.p','rb'))
    # print('-----'*8)
    it = 0
    all_acc = []
    all_r_acc = []
    for res_str in results:
        # print(res_str)
        if res_str.startswith('Accuracy on testset:'):
            acc = float(res_str[len('Accuracy on testset: '):len('Accuracy on testset: ')+5])
            all_acc.append(acc)

        if res_str.startswith('Went from'):
            start_batch = 391
            end_batch = int(res_str[len('Went from 391 batches to'):len('Went from 391 batches to ')+3])
            r_acc = 100 - ((end_batch-start_batch)/start_batch)*100
            print('iter %d: %.2f/%.2f'%(it,r_acc,acc))
            it += 1
            all_r_acc.append(r_acc)

    all_r_acc = np.array(all_r_acc)
    all_acc = np.array(all_acc)
    print('-----'*8)
    print('iter %d: %.2f'%(np.argmax(all_r_acc),np.max(all_r_acc)))
    print('iter %d: %.2f'%(np.argmax(all_acc),np.max(all_acc)))
    print('-----'*8)
    acc_data.append(all_acc)
    r_acc_data.append(all_r_acc)

acc_data = np.asarray(acc_data)
r_acc_data = np.asarray(r_acc_data)

Evaluating Clip
iter 0: 49.36/81.89
iter 1: 60.87/76.77
iter 2: 53.71/79.62
iter 3: 69.57/82.92
iter 4: 53.45/83.80
iter 5: 61.89/67.75
iter 6: 53.96/75.60
iter 7: 78.01/86.34
iter 8: 60.87/84.34
----------------------------------------
iter 7: 78.01
iter 7: 86.34
----------------------------------------
iter 0: 43.99/81.75
iter 1: 70.59/72.23
iter 2: 70.33/84.37
iter 3: 66.50/84.83
iter 4: 54.48/79.97
iter 5: 53.71/77.51
iter 6: 58.57/76.54
iter 7: 67.26/85.69
iter 8: 55.50/85.12
----------------------------------------
iter 1: 70.59
iter 9: 85.78
----------------------------------------
iter 0: 47.31/82.19
iter 1: 65.98/84.23
iter 2: 54.99/85.70
iter 3: 52.69/71.43
iter 4: 63.94/83.42
iter 5: 61.13/84.32
iter 6: 61.64/85.75
iter 7: 53.45/85.98
iter 8: 50.64/77.04
----------------------------------------
iter 1: 65.98
iter 9: 86.20
----------------------------------------
iter 0: 44.50/82.20
iter 1: 60.10/70.69
iter 2: 57.03/81.47
iter 3: 61.13/82.43
iter 4: 59.34/84.88
iter 5: 57.29/

In [60]:
print('acc: ',acc_data.mean(axis=0))
print('r_acc: ',r_acc_data.mean(axis=0))

acc:  [81.974 75.05  82.092 80.988 83.496 75.988 81.552 84.106 81.314 85.764]
r_acc:  [45.98465473 64.60358056 58.00511509 63.47826087 59.07928389 57.69820972
 59.64194373 63.1202046  57.69820972]


In [61]:
print('Evaluating Simple')
acc_data = []
r_acc_data = []
for path in subdirs_simple:
    results = pickle.load(open(path[0:-len('setup.json')]+'results.p','rb'))
    f_results = pickle.load(open(path[0:-len('setup.json')]+'f_results.p','rb'))
    # print('-----'*8)
    it = 0
    all_acc = []
    all_r_acc = []
    for res_str in results:
        # print(res_str)
        if res_str.startswith('Accuracy on testset:'):
            acc = float(res_str[len('Accuracy on testset: '):len('Accuracy on testset: ')+5])
            all_acc.append(acc)

        if res_str.startswith('Went from'):
            start_batch = 391
            end_batch = int(res_str[len('Went from 391 batches to'):len('Went from 391 batches to ')+3])
            r_acc = 100 - ((end_batch-start_batch)/start_batch)*100
            print('iter %d: %.2f/%.2f'%(it,r_acc,acc))
            it += 1
            all_r_acc.append(r_acc)

    all_r_acc = np.array(all_r_acc)
    all_acc = np.array(all_acc)
    print('-----'*8)
    print('iter %d: %.2f'%(np.argmax(all_r_acc),np.max(all_r_acc)))
    print('iter %d: %.2f'%(np.argmax(all_acc),np.max(all_acc)))
    print('-----'*8)
    acc_data.append(all_acc)
    r_acc_data.append(all_r_acc)

acc_data = np.asarray(acc_data)
r_acc_data = np.asarray(r_acc_data)

Evaluating Simple
iter 0: 58.31/78.98
iter 1: 72.38/77.54
iter 2: 72.38/70.53
iter 3: 70.08/74.68
iter 4: 78.26/82.55
iter 5: 77.49/80.94
iter 6: 76.98/79.21
iter 7: 79.03/80.83
iter 8: 71.36/67.34
----------------------------------------
iter 7: 79.03
iter 4: 82.55
----------------------------------------
iter 0: 59.85/79.47
iter 1: 68.03/72.86
iter 2: 72.12/78.98
iter 3: 78.77/79.73
iter 4: 73.15/81.83
iter 5: 70.08/76.26
iter 6: 73.15/82.30
iter 7: 79.28/81.88
iter 8: 66.50/70.13
----------------------------------------
iter 7: 79.28
iter 6: 82.30
----------------------------------------
iter 0: 59.08/77.78
iter 1: 70.33/73.17
iter 2: 74.42/82.22
iter 3: 70.59/69.75
iter 4: 71.87/78.72
iter 5: 76.98/81.13
iter 6: 77.24/81.63
iter 7: 80.82/81.45
iter 8: 79.80/81.22
----------------------------------------
iter 7: 80.82
iter 2: 82.22
----------------------------------------
iter 0: 59.85/80.22
iter 1: 71.61/69.49
iter 2: 69.31/67.42
iter 3: 79.03/82.48
iter 4: 69.05/75.19
iter 5: 72.3

In [62]:
print('acc: ',acc_data.mean(axis=0))
print('r_acc: ',r_acc_data.mean(axis=0))

acc:  [79.24  71.272 76.376 74.074 80.1   78.644 80.24  78.648 74.526 73.942]
r_acc:  [59.18158568 70.58823529 72.42966752 73.55498721 73.24808184 74.42455243
 74.47570332 76.8286445  71.04859335]


In [5]:
path_simple = '/home/mbeliaev/home/code/robust-l0/new_trained/beta_100_no/MNIST/cnn_small/simple'

subdirs_simple = [os.path.join(root, name)
             for root, dirs, files in os.walk(path_simple)
             for name in files
             if name.endswith((".json"))]

print(subdirs_simple)




['/home/mbeliaev/home/code/robust-l0/new_trained/beta_100_no/MNIST/cnn_small/simple/k_12/adv_12/5/setup.json', '/home/mbeliaev/home/code/robust-l0/new_trained/beta_100_no/MNIST/cnn_small/simple/k_12/adv_12/3/setup.json', '/home/mbeliaev/home/code/robust-l0/new_trained/beta_100_no/MNIST/cnn_small/simple/k_12/adv_12/4/setup.json', '/home/mbeliaev/home/code/robust-l0/new_trained/beta_100_no/MNIST/cnn_small/simple/k_12/adv_12/2/setup.json', '/home/mbeliaev/home/code/robust-l0/new_trained/beta_100_no/MNIST/cnn_small/simple/k_12/adv_12/1/setup.json']


In [7]:
print('Evaluating Simple')
acc_data = []
r_acc_data = []
for path in subdirs_simple:
    results = pickle.load(open(path[0:-len('setup.json')]+'results.p','rb'))
    f_results = pickle.load(open(path[0:-len('setup.json')]+'f_results.p','rb'))
    # print('-----'*8)
    it = 0
    all_acc = []
    all_r_acc = []
    for res_str in results:
        print(res_str)
        if res_str.startswith('Accuracy on testset:'):
            acc = float(res_str[len('Accuracy on testset: '):len('Accuracy on testset: ')+5])
            all_acc.append(acc)

        if res_str.startswith('Went from'):
            start_batch = 391
            end_batch = int(res_str[len('Went from 391 batches to'):len('Went from 391 batches to ')+3])
            r_acc = 100 - ((end_batch-start_batch)/start_batch)*100
            print('iter %d: %.2f/%.2f'%(it,r_acc,acc))
            it += 1
            all_r_acc.append(r_acc)

#     all_r_acc = np.array(all_r_acc)
#     all_acc = np.array(all_acc)
#     print('-----'*8)
#     print('iter %d: %.2f'%(np.argmax(all_r_acc),np.max(all_r_acc)))
#     print('iter %d: %.2f'%(np.argmax(all_acc),np.max(all_acc)))
#     print('-----'*8)
#     acc_data.append(all_acc)
#     r_acc_data.append(all_r_acc)

# acc_data = np.asarray(acc_data)
# r_acc_data = np.asarray(r_acc_data)

Evaluating Simple
Running iter 0
Accuracy on testset: 11.350
Accuracy on trainset: 11.237
Running iter 1
Accuracy on testset: 11.350
Accuracy on trainset: 11.237
Running iter 2
Accuracy on testset: 11.350
Accuracy on trainset: 11.237
Running iter 3
Accuracy on testset: 11.350
Accuracy on trainset: 11.237
Running iter 4
Accuracy on testset: 11.350
Accuracy on trainset: 11.237
Running iter 5
Accuracy on testset: 11.350
Accuracy on trainset: 11.237
Running iter 6
Accuracy on testset: 11.350
Accuracy on trainset: 11.237
Running iter 7
Accuracy on testset: 11.350
Accuracy on trainset: 11.237
Running iter 8
Accuracy on testset: 11.350
Accuracy on trainset: 11.237
Running iter 9
Accuracy on testset: 11.350
Accuracy on trainset: 11.237
Final testset accuracy: 11.350
Will measure accuracy after training
Running iter 0
Accuracy on testset: 11.350
Accuracy on trainset: 11.237
Running iter 1
Accuracy on testset: 11.350
Accuracy on trainset: 11.237
Running iter 2
Accuracy on testset: 10.100
Accurac

In [ ]:
print('acc: ',acc_data.mean(axis=0))
print('r_acc: ',r_acc_data.mean(axis=0))

In [4]:
cifar = prep_data(root,128,'CIFAR')

Files already downloaded and verified
Files already downloaded and verified


In [15]:
device = 'cuda:2'
# Get the clean accuracy or 
mnist_clip_path = '/home/mbeliaev/home/code/robust-l0/new_trained/new_adv/MNIST/cnn_small/clip/k_12/adv_12/99/'
mnist_simp_path = '/home/mbeliaev/home/code/robust-l0/new_trained/new_adv/MNIST/cnn_small/simple/k_12/adv_12/99/'
mnist_conv_path = '/home/mbeliaev/home/code/robust-l0/new_trained/newest_adv/MNIST/cnn_small/conv/k_12/adv_12/99/'
# paths
cifar_clip_path = '/home/mbeliaev/home/code/robust-l0/new_trained/new_adv/CIFAR/VGG16/clip/k_12/adv_12/99/'
cifar_simp_path = '/home/mbeliaev/home/code/robust-l0/new_trained/new_adv/CIFAR/VGG16/simple/k_12/adv_12/99/'
cifar_conv_path = '/home/mbeliaev/home/code/robust-l0/new_trained/newest_adv/CIFAR/VGG16/conv/k_12/adv_12/99'

In [11]:
# evaluate accuracy and r acc
def test_model(net_path, x, y, k, perturb, n_queries,  device):
    # first load the x, y
    input_shape = x[0].shape
    config = json.load(open(net_path+'/setup.json','rb'))

    net = Net(cfg_name=config['cfg_name'],
        k = k,
        input_shape=input_shape,
        trunc_type=config['trunc_type'])

    net.to(device)
    net.load_state_dict(torch.load(net_path+'/net.pth', map_location=device))
    net.eval()

    # next evaluate accuracy
    acc = evaluate(net, x, y, device)
    print("Clean accuracy of netwrk: %.2f"%acc,"%")

    # and robust accuracy
    r_acc, _, _, _ = attack(net,perturb,x,y,device=device,n_queries=n_queries,n_restarts=1,log_path='log.txt')
    print("Robust accuracy of netwrk: %.2f"%r_acc,"%")


In [13]:
batches = 1
n_queries = 50
perturb = 12

methods = ["Clip", "Weighted Clip", "Conv"]
paths = [mnist_clip_path,mnist_simp_path,mnist_conv_path]

print("MNIST EVAL")
for i_method in range(len(methods)):
    print(methods[i_method], "method")
    print("-----"*8)
    for k in [5,12,24,50]:
        print("k=",k)
        test_model(paths[i_method], mnist['x_test'][0:batches],mnist['y_test'][0:batches],k,perturb,n_queries,device)

MNIST EVAL
clip method
----------------------------------------
Clip method
k= 5
Clean accuracy of netwrk: 98.44 %
Robust accuracy of netwrk: 82.81 %hes done
k= 12
Clean accuracy of netwrk: 98.05 %
Robust accuracy of netwrk: 95.70 %hes done
k= 24
Clean accuracy of netwrk: 97.66 %
Robust accuracy of netwrk: 95.31 %hes done
k= 50
Clean accuracy of netwrk: 87.89 %
Robust accuracy of netwrk: 78.91 %hes done
Weighted Clip method
k= 5
Clean accuracy of netwrk: 97.27 %
Robust accuracy of netwrk: 75.39 %hes done
k= 12
Clean accuracy of netwrk: 96.48 %
Robust accuracy of netwrk: 78.12 %hes done
k= 24
Clean accuracy of netwrk: 94.53 %
Robust accuracy of netwrk: 80.86 %hes done
k= 50
Clean accuracy of netwrk: 79.69 %
Robust accuracy of netwrk: 60.94 %hes done
Conv method
k= 5
Clean accuracy of netwrk: 98.83 %
Robust accuracy of netwrk: 76.17 %hes done
k= 12
Clean accuracy of netwrk: 98.05 %
Robust accuracy of netwrk: 84.38 %hes done
k= 24
Clean accuracy of netwrk: 98.44 %
Robust accuracy of netwr

In [18]:
batches = 5
n_queries = 100
perturb = 12

methods = ["Clip", "Weighted Clip", "Conv"]
paths = [cifar_clip_path,cifar_simp_path,cifar_conv_path]

print("CIFAR EVAL")
for i_method in range(len(methods)):
    print(methods[i_method], "method")
    print("-----"*8)
    for k in [8,12,16,20,24]:
        print("k=",k)
        test_model(paths[i_method], cifar['x_test'][0:batches],cifar['y_test'][0:batches],k,perturb,n_queries,device)

CIFAR EVAL
Clip method
----------------------------------------
k= 8
Clean accuracy of netwrk: 78.44 %
single channel mode
Robust accuracy of netwrk: 54.53 %hes done
k= 12
Clean accuracy of netwrk: 78.28 %
single channel mode
Robust accuracy of netwrk: 54.53 %hes done
k= 16
Clean accuracy of netwrk: 78.12 %
single channel mode
Robust accuracy of netwrk: 53.91 %hes done
k= 20
Clean accuracy of netwrk: 77.50 %
single channel mode
Robust accuracy of netwrk: 51.41 %hes done
k= 24
Clean accuracy of netwrk: 77.03 %
single channel mode
Robust accuracy of netwrk: 53.91 %hes done
Weighted Clip method
----------------------------------------
k= 8
Clean accuracy of netwrk: 83.28 %
single channel mode
Robust accuracy of netwrk: 67.97 %hes done
k= 12
Clean accuracy of netwrk: 83.44 %
single channel mode
Robust accuracy of netwrk: 68.91 %hes done
k= 16
Clean accuracy of netwrk: 83.44 %
single channel mode
Robust accuracy of netwrk: 69.38 %hes done
k= 20
Clean accuracy of netwrk: 83.44 %
single chann

In [3]:
# FC No adversary results
trunc_type = ['clip', 'simple']
# paths_5 = ['/home/mbeliaev/home/code/robust-l0/new_trained/no_adv/MNIST/mlp/%s/k_5/adv_5/99/'%trunc for trunc in trunc_type]
paths_12 = ['/home/mbeliaev/home/code/robust-l0/new_trained/new_no_adv/MNIST/cnn_small/%s/k_12/adv_12/99/'%trunc for trunc in trunc_type]
paths_og = ['/home/mbeliaev/home/code/robust-l0/new_trained/new_no_adv/MNIST/cnn_small/clip/k_0/adv_%d/99/'%adv for adv in [12]]

# adv_paths_5 = ['/home/mbeliaev/home/code/robust-l0/new_trained/adv/MNIST/mlp/%s/k_5/adv_5/99/'%trunc for trunc in trunc_type]
adv_paths_12 = ['/home/mbeliaev/home/code/robust-l0/new_trained/new_adv/MNIST/cnn_small/%s/k_12/adv_12/99/'%trunc for trunc in trunc_type]
adv_paths_og = ['/home/mbeliaev/home/code/robust-l0/new_trained/new_adv/MNIST/cnn_small/clip/k_0/adv_%d/99/'%adv for adv in [12]]

keys = ['k', 'perturb']
print("Original CNN Network")
print("-----"*8)
for path, adv_path in zip(paths_og,adv_paths_og):
    # show configuration
    config = json.load(open(path+'setup.json','rb'))
    config_adv = json.load(open(adv_path+'setup.json','rb'))
    print('Configuration:')
    for key in keys:
        print('\t', key,': ', config[key],config_adv[key])
    print("-----"*8)
    f_res = pickle.load(open(path+'/f_results.p','rb'))
    print('NO ADV Acc: %.2f, R. Acc: %.2f'%(f_res[0],f_res[1]))
    f_res = pickle.load(open(adv_path+'/f_results.p','rb'))
    print('ADV    Acc: %.2f, R. Acc: %.2f'%(f_res[0],f_res[1]))
    print("-----"*8)

print("Truncated CNN Networks")
keys = ['trunc_type','k', 'perturb']

for path, adv_path in zip(paths_12,adv_paths_12):
    # show configuration
    config = json.load(open(path+'setup.json','rb'))
    config_adv = json.load(open(adv_path+'setup.json','rb'))
    print('Configuration:')
    for key in keys:
        print('\t', key,': ', config[key],config_adv[key])
    print("-----"*8)
    f_res = pickle.load(open(path+'/f_results.p','rb'))
    print('NO ADV Acc: %.2f, R. Acc: %.2f'%(f_res[0],f_res[1]))
    f_res = pickle.load(open(adv_path+'/f_results.p','rb'))
    print('ADV    Acc: %.2f, R. Acc: %.2f'%(f_res[0],f_res[1]))
    print("-----"*8)


Original CNN Network
----------------------------------------
Configuration:
	 k :  0 0
	 perturb :  12 12
----------------------------------------
NO ADV Acc: 98.25, R. Acc: 0.14
ADV    Acc: 98.48, R. Acc: 3.35
----------------------------------------
Truncated CNN Networks
Configuration:
	 trunc_type :  clip clip
	 k :  12 12
	 perturb :  12 12
----------------------------------------
NO ADV Acc: 98.14, R. Acc: 36.26
ADV    Acc: 98.25, R. Acc: 55.78
----------------------------------------
Configuration:
	 trunc_type :  simple simple
	 k :  12 12
	 perturb :  12 12
----------------------------------------
NO ADV Acc: 98.09, R. Acc: 0.02
ADV    Acc: 97.69, R. Acc: 4.43
----------------------------------------


In [4]:
# FC No adversary results
trunc_type = ['clip', 'simple']
# paths_5 = ['/home/mbeliaev/home/code/robust-l0/new_trained/no_adv/MNIST/mlp/%s/k_5/adv_5/99/'%trunc for trunc in trunc_type]
paths_12 = ['/home/mbeliaev/home/code/robust-l0/new_trained/new_no_adv/CIFAR/VGG16/%s/k_12/adv_12/99/'%trunc for trunc in trunc_type]
paths_og = ['/home/mbeliaev/home/code/robust-l0/new_trained/new_no_adv/CIFAR/VGG16/clip/k_0/adv_%d/99/'%adv for adv in [12]]

# adv_paths_5 = ['/home/mbeliaev/home/code/robust-l0/new_trained/adv/MNIST/mlp/%s/k_5/adv_5/99/'%trunc for trunc in trunc_type]
adv_paths_12 = ['/home/mbeliaev/home/code/robust-l0/new_trained/new_adv/CIFAR/VGG16/%s/k_12/adv_12/99/'%trunc for trunc in trunc_type]
adv_paths_og = ['/home/mbeliaev/home/code/robust-l0/new_trained/new_adv/CIFAR/VGG16/clip/k_0/adv_%d/99/'%adv for adv in [12]]

keys = ['k', 'perturb']
print("Original VGG Network")
print("-----"*8)
for path, adv_path in zip(paths_og,adv_paths_og):
    # show configuration
    config = json.load(open(path+'setup.json','rb'))
    config_adv = json.load(open(adv_path+'setup.json','rb'))
    print('Configuration:')
    for key in keys:
        print('\t', key,': ', config[key],config_adv[key])
    print("-----"*8)
    f_res = pickle.load(open(path+'/f_results.p','rb'))
    print('NO ADV Acc: %.2f, R. Acc: %.2f'%(f_res[0],f_res[1]))
    f_res = pickle.load(open(adv_path+'/f_results.p','rb'))
    print('ADV    Acc: %.2f, R. Acc: %.2f'%(f_res[0],f_res[1]))
    print("-----"*8)

print("Truncated VGG Networks")
keys = ['trunc_type','k', 'perturb']

for path, adv_path in zip(paths_12,adv_paths_12):
    # show configuration
    config = json.load(open(path+'setup.json','rb'))
    config_adv = json.load(open(adv_path+'setup.json','rb'))
    print('Configuration:')
    for key in keys:
        print('\t', key,': ', config[key],config_adv[key])
    print("-----"*8)
    f_res = pickle.load(open(path+'/f_results.p','rb'))
    print('NO ADV Acc: %.2f, R. Acc: %.2f'%(f_res[0],f_res[1]))
    f_res = pickle.load(open(adv_path+'/f_results.p','rb'))
    print('ADV    Acc: %.2f, R. Acc: %.2f'%(f_res[0],f_res[1]))
    print("-----"*8)

Original VGG Network
----------------------------------------
Configuration:
	 k :  0 0
	 perturb :  12 12
----------------------------------------
NO ADV Acc: 87.03, R. Acc: 0.19
ADV    Acc: 86.53, R. Acc: 3.24
----------------------------------------
Truncated VGG Networks
Configuration:
	 trunc_type :  clip clip
	 k :  12 12
	 perturb :  12 12
----------------------------------------
NO ADV Acc: 86.00, R. Acc: 12.54
ADV    Acc: 77.43, R. Acc: 15.14
----------------------------------------
Configuration:
	 trunc_type :  simple simple
	 k :  12 12
	 perturb :  12 12
----------------------------------------
NO ADV Acc: 82.84, R. Acc: 17.13
ADV    Acc: 81.18, R. Acc: 27.43
----------------------------------------


In [8]:
# # CIFAR results
# trunc_type = ['clip','simple']
# seeds = [99]
# paths_12 = ['/home/mbeliaev/home/code/robust-l0/new_trained/new_no_adv/CIFAR/VGG16/%s/k_12/adv_12/'%trunc for trunc in trunc_type]
# paths_og = ['/home/mbeliaev/home/code/robust-l0/new_trained/new_no_adv/CIFAR/VGG16/conv/k_0/adv_%d/'%adv for adv in [12]]

# VGG16paths_12 = ['/home/mbeliaev/home/code/robust-l0/new_trained/new_adv/CIFAR/VGG16/%s/k_12/adv_12/'%trunc for trunc in trunc_type]
# VGG16paths_og = ['/home/mbeliaev/home/code/robust-l0/new_trained/new_adv/CIFAR/VGG16/clip/k_0/adv_%d/'%adv for adv in [12]]

# keys = ['trunc_type','k', 'perturb']
# print("Original CNN Network")
# print("-----"*8)
# for path, adv_path in zip(paths_og,adv_paths_og):
#     # show configuration
#     config = json.load(open(path+'99/setup.json','rb'))
#     config_adv = json.load(open(adv_path+'99/setup.json','rb'))
#     print('Configuration:')
#     for key in keys:
#         print('\t', key,': ', config[key],config_adv[key])
#     print("-----"*8)
#     acc = 0
#     r_acc = 0
#     for seed in seeds:
#         f_res = pickle.load(open(path+'%d/f_results.p'%seed,'rb'))
#         acc += f_res[0]
#         r_acc += f_res[1]
#     acc /= len(seeds)
#     r_acc /= len(seeds)

#     print('NO ADV Acc: %.2f, R. Acc: %.2f'%(acc,r_acc))

#     acc = 0
#     r_acc = 0
#     for seed in seeds:
#         f_res = pickle.load(open(adv_path+'%d/f_results.p'%seed,'rb'))
#         acc += f_res[0]
#         r_acc += f_res[1]
#     acc /= len(seeds)
#     r_acc /= len(seeds)
#     print('ADV    Acc: %.2f, R. Acc: %.2f'%(acc,r_acc))
#     print("-----"*8)

# print("Truncated CNN Network")
# for path, adv_path in zip(paths_12,adv_paths_12):
#     # show configuration
#     config = json.load(open(path+'99/setup.json','rb'))
#     config_adv = json.load(open(adv_path+'99/setup.json','rb'))
#     print('Configuration:')
#     for key in keys:
#         print('\t', key,': ', config[key],config_adv[key])
#     print("-----"*8)
#     acc = 0
#     r_acc = 0
#     for seed in seeds:
#         f_res = pickle.load(open(path+'%d/f_results.p'%seed,'rb'))
#         acc += f_res[0]
#         r_acc += f_res[1]
#     acc /= len(seeds)
#     r_acc /= len(seeds)

#     print('NO ADV Acc: %.2f, R. Acc: %.2f'%(acc,r_acc))

#     acc = 0
#     r_acc = 0
#     for seed in seeds:
#         f_res = pickle.load(open(adv_path+'%d/f_results.p'%seed,'rb'))
#         acc += f_res[0]
#         r_acc += f_res[1]
#     acc /= len(seeds)
#     r_acc /= len(seeds)
#     print('ADV    Acc: %.2f, R. Acc: %.2f'%(acc,r_acc))
#     print("-----"*8)


Original CNN Network
----------------------------------------
Configuration:
	 trunc_type :  conv conv
	 k :  0 0
	 perturb :  12 12
----------------------------------------
NO ADV Acc: 98.51, R. Acc: 0.02
ADV    Acc: 98.51, R. Acc: 6.09
----------------------------------------
Truncated CNN Network
Configuration:
	 trunc_type :  clip clip
	 k :  12 12
	 perturb :  12 12
----------------------------------------
NO ADV Acc: 98.52, R. Acc: 3.28
ADV    Acc: 98.58, R. Acc: 26.93
----------------------------------------
Configuration:
	 trunc_type :  clip_abs clip_abs
	 k :  12 12
	 perturb :  12 12
----------------------------------------
NO ADV Acc: 98.56, R. Acc: 0.00
ADV    Acc: 98.27, R. Acc: 24.11
----------------------------------------
Configuration:
	 trunc_type :  conv conv
	 k :  12 12
	 perturb :  12 12
----------------------------------------
NO ADV Acc: 98.33, R. Acc: 3.07
ADV    Acc: 98.37, R. Acc: 11.76
----------------------------------------
